In [ ]:
# making utils as python module so as to import user defined functions
import sys
sys.path.append('../utils')

# Importing libraries
import re
import pandas as pd
import numpy as np
import seaborn as sn
from scipy import stats
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from MyFunctions import show_counts, count_num_nonNum, fixing_author, count_null_value, null_counts_pie_chart, clean_location, extract_image

## Data Collection

In [ ]:
# Reading csv files
book_df=pd.read_csv('../data/raw/Books.csv',low_memory=False)
ratings_df=pd.read_csv('../data/raw/Ratings.csv')
users_df=pd.read_csv('../data/raw/Users.csv')

## Exploratory Data Analysis:

In [ ]:
# Checking for duplicate vaues
print('book_df   :',book_df.duplicated().sum())
print('ratings_df:',ratings_df.duplicated().sum())
print('users_df  :',users_df.duplicated().sum())

0 duplicate entries

In [ ]:
# Checking shape of DataFrames
print('book_df   :',book_df.shape)
print('ratings_df:',ratings_df.shape)
print('users_df  :',users_df.shape)

#### 1. book_df

Can See Some DtypeWarning

In [ ]:
# Checking how the dataframe looks like
book_df.sample(5)

In [ ]:
# gaining info of the columns
book_df.info(memory_usage='deep')

 All columns are defied as 'Object' data type which is memory consuming.<br>
To fix this we have to check all the values first

In [ ]:
# How many numerical and non numerical values present inside it
show_counts(book_df)

* ISBN can be a string data type
* Book title always be of string data type


In [ ]:
# In 'Book-Title' column, we can see 38 numeric values
# Lets find out them

book_df.iloc[count_num_nonNum(book_df['Book-Title'],return_num_index=True),].head()

Seems like these are legit

In [ ]:
# Lets check Book-Author and see why there are 3 integer numbers
temp_index = count_num_nonNum(book_df['Book-Author'],return_num_index=True)         # To store index

book_df.iloc[temp_index,]

except 'ISBN' and 'Book-Title', all columns are missplaced<br>
* In 'Book-Title' after   \\"; it is the authors name

In [ ]:
# First fixing all columns repositioning all values
book_df.iloc[temp_index,3:]=book_df.iloc[temp_index,2:-1]

In [ ]:
pd.set_option('display.max_colwidth', None)     # To read full text

In [ ]:
# Showing the changes
book_df.iloc[temp_index]

Now time to change author name

In [ ]:
# Cehcking if any book published by the author 'Jean-Marie Gustave Le ClÃ?Â©zio'
book_df[book_df['Book-Author']=='Jean-Marie Gustave Le ClÃ?Â©zio']

In [ ]:
# Actual name of the author is 'Jean-Marie Gustave Le Clézio'
book_df[book_df['Book-Author']=='Jean-Marie Gustave Le Clézio']

We can change the name but it's not necessary here


In [ ]:
# Fixing 'Book-Author' column
temp_stor=book_df.iloc[temp_index,1].apply(fixing_author)

In [ ]:
# storing values to both columns
book_df.iloc[temp_index,1]=temp_stor.apply(lambda x:x[0])
book_df.iloc[temp_index,2]=temp_stor.apply(lambda x:x[1])

In [ ]:
book_df.iloc[temp_index]

In [ ]:
# Checking if all column got fixed or not
show_counts(book_df)

In [ ]:
# Checking publisher column
book_df.iloc[count_num_nonNum(book_df['Publisher'],return_num_index=True)]

In [ ]:
# All of these above books are published by 10/18
# Replacing them with '10/18'
book_df.iloc[count_num_nonNum(book_df['Publisher'],return_num_index=True),4]='10/18'

In [ ]:
# Can see some null values present
# Checking them
count_null_value(book_df)

In [ ]:
sn.heatmap(book_df.isnull(),cbar=False)

In [ ]:
# Book-Author and Publisher has some null values
book_df[book_df['Book-Author'].isnull()]

In [ ]:
# Could not find the author name so
book_df.loc[187689,'Book-Author']='Sophie Pyott'

In [ ]:
book_df[book_df['Publisher'].isnull()]

In [ ]:
# Both of these books are published by 'Novelbooks'
book_df['Publisher'].fillna('Novelbooks',inplace=True)

In [ ]:
# In our dataset, we are given ISBN-10 (Not ISBN-13).
# ISBN only contains numbers and sometimes last number as 'X'
# Also its length is 10

book_df['ISBN'].apply(lambda x: len(x)).value_counts()

In [ ]:
# Some are of length 13 and 11
# Lets see who are they

book_df.loc[book_df[book_df['ISBN'].apply(lambda x: len(x)!=10)].index]

In [ ]:
# First 10 letters of the string are valid ISBN
book_df['ISBN']=book_df['ISBN'].apply(lambda x: x.strip().upper()[:10])

In [ ]:
# Again checking datatypes
book_df.info()

In [ ]:
#'Year-Of-Publication' should be 'int' type isn't it !
book_df['Year-Of-Publication']=book_df['Year-Of-Publication'].astype('int16')

In [ ]:
# Got a problem with 'Year-Of-Publication'
np.sort(book_df['Year-Of-Publication'].unique())


There are some 0 values present, which is not possible<br>
Also 2024!  (A book from the Future!)


In [ ]:
# Box plot for 'Year-Of-Publication'
go.Figure(go.Box(x=book_df['Year-Of-Publication']))

<pre>
we will consider those books after year '1806' and before '2023'
You might wonder why 1806?
If u see all unique years all are in a range of 1800-1900-2000.
But 1300 thats 500 years later from the trend.
Values which differs significantly from other observations are called outliers right
Thats how thses are also outliers</pre>

In [ ]:

book_df.loc[book_df[(book_df['Year-Of-Publication']<1800) | (book_df['Year-Of-Publication']>2023.0)].index,'Year-Of-Publication']=np.nan

In [ ]:
# Null Counts
book_df['Year-Of-Publication'].isnull().sum()

In [ ]:
# visualizing what percentage values are null
fig = null_counts_pie_chart(book_df['Year-Of-Publication'])
fig.show()

# saving as html file
fig.write_html("../Charts/null_year_before_scrapping.html")

In [ ]:
# To fill those null values we need web scrapping
# so dumping book_df
book_df.to_csv('../data/processed/book_df_before_scrapping.csv',index=False)

In [ ]:
# reading the scrapped file
book_df = pd.read_csv('../data/processed/book_df_after_scrapping.csv', low_memory=False)

In [ ]:
# Again Checking all unique values
np.sort(book_df['Year-Of-Publication'].unique())

<pre>
After scrapping we have can see some values from 13th centuries, also years after 2023.
So we conclude that even website are not providing accurate information.
</pre>


In [ ]:
# Still there are some values wihch is greater than '2023'
# This dataset is 2 yaers old i.e from 2021.
book_df.loc[book_df[book_df['Year-Of-Publication']>2021.0].index,'Year-Of-Publication']=np.nan

In [ ]:
# Checking How many null values left
count_null_value(book_df)

In [ ]:
# Checking what percentage values are null
null_counts_pie_chart(book_df['Year-Of-Publication'])
fig.show()

# saving as html file
fig.write_html("../Charts/null_year_after_scrapping.html")

With the help of web scrapping we reduced our null values from 1.71% to 0.0479% <br>
Should we drop them or impute with mode value?<br>
Lets find out

In [ ]:
# Checking the distribution of 'Year-Of-Publication'
sn.histplot(book_df['Year-Of-Publication'])

After the Year '1950' an exponentially growth of book published can be shown.

In [ ]:
# With 'mode' values transformation

temp_year=book_df['Year-Of-Publication'].dropna()
temp_year_mode_imputed=book_df['Year-Of-Publication'].fillna(stats.mode(temp_year)[0])
plt.figure(figsize=(18,6))
plt.subplot(1,2,1)
sn.histplot(temp_year,
            kde=True,
            color='#fcb603',
            label=f"Skewness: {stats.skew(temp_year)}\nKurtosis: {stats.kurtosis(temp_year)}")
plt.legend()
plt.subplot(1,2,2)
sn.histplot(temp_year_mode_imputed,
            kde=True,
            color='#03fc28',
            label=f"After 'Mode' imputation\nSkewness: {stats.skew(temp_year_mode_imputed)}\nKurtosis: {stats.kurtosis(temp_year_mode_imputed)}")
plt.legend()

In [ ]:
# Note, in 'ISBN' and 'Book-Title' columns we should not have repeated values right!
# But...

print(f"In 'ISBN' column we have {book_df['ISBN'].duplicated().sum()} and \nin 'Book-Title' we have {book_df['Book-Title'].duplicated().sum()} no of duplicate values")

In [ ]:
# Lets find out
book_df['Book-Title'].value_counts()

In [ ]:
# Checking those rows where 'Book-Title' is 'Selected Poems'
book_df[book_df['Book-Title']=='Selected Poems'].head()

I might wrong here, book title can be same while with a different ISBN no as, Author, Published Year and Publisher can be different.

Incase a string typed with different cases (i.e upper and lower case)

In [ ]:
# differce of unique values after converting them to lower case
book_df['Book-Title'].apply(lambda x:x.strip()).nunique()-book_df['Book-Title'].apply(lambda x:x.lower().strip()).nunique()

In [ ]:
# Finding what are they
a=book_df['Book-Title'].apply(lambda x:x.strip()).unique()                # Storing all unique values
a=pd.DataFrame(pd.Series(a).apply(lambda x:x.lower()).value_counts())     # count frequencies with lower case letters
a[a>1].head()       # only those which are repeated more than once

In [ ]:
# Lets see how book title named 'le vieil homme et la mer' are typed differently
book_df[book_df['Book-Title'].apply(lambda x:x.lower().strip())=='le vieil homme et la mer']

In last row except 'Le' all are written in lowercase letters, <br>
and in second row, 'homme et la' is in lowercase letters.

In [ ]:
book_df[['Book-Title', 'Book-Author', 'Publisher']].head()

In [ ]:
# We can do lowercase them all.
# Trying something new with 'Title-Case'.
# Fixing 'Book-Title', 'Book-Author' and 'Publisher' columns

for i in ['Book-Title', 'Book-Author', 'Publisher']:
    book_df.loc[:, i] = book_df.loc[:, i].str.title()

In [ ]:

# Here is the result
book_df[['Book-Title', 'Book-Author', 'Publisher']].head()

Noticed some values are 'Not Applicable (Na )' in Author's name.<br>

In general, the use of "N/A" or "Not Applicable" in the author field of a book record is intended<br>  to indicate that there is no relevant information available or applicable for that field, and should <br>not be interpreted as an error or omission.

In [ ]:
temp_na_df=book_df[book_df['Book-Author']=='Not Applicable (Na )']
temp_na_df.head(3)

In [ ]:
# Checking how rows have these values
fig=go.Figure(go.Pie(labels=['Unknown Authors Count','Known to us'],
                    values=[len(temp_na_df),len(book_df)-len(temp_na_df)],
                    marker=dict(colors=['#ff0303','#77f7a6']),
                    pull=[0,0.1]))
fig.update_layout(title="Unknown Authors",
                    autosize=False,)
fig.show()

In [ ]:
# we can replace them with 'unknown' or 'others' or 'nan'.
# But I will go with 'others'

book_df.loc[temp_na_df.index,'Book-Author']='others'

In [ ]:
# Noticed '\' in 'Book-Title' we should replace them

book_df['Book-Title']=book_df['Book-Title'].apply(lambda x: x.replace('\\',''))

* Top 20 Authors who have written the most no of books.

In [ ]:
temp_df=book_df['Book-Author'].value_counts()[:20]

fig = go.Figure()
fig.add_bar(x=temp_df.index,
            y=temp_df.values,
            text=temp_df.index,
            hovertemplate ='Total Books:<br> %{y}<extra></extra>',
            marker=dict(color=temp_df.values,
                        colorscale='Darkmint'),
            marker_line_width=1.,
            width=0.6,
            marker_line_color='#02b0fa',)
annotations=[dict(x=val[0], y=val[1]+20, text=str(val[1]), showarrow=False) for val in zip(temp_df.index, temp_df.values)]
fig.update_layout(annotations=annotations,
                  template="plotly_white",
                  title='<b>Author with most written books<b>',
                  margin=dict(t=65),
                  xaxis=dict(
                            showgrid=False,
                            linecolor='#728082',
                            showticklabels=False,),
                  yaxis=dict(
                            showgrid=False,
                            title='No of Books',
                            linecolor='#728082',
                            ticks='outside'),)
# saving as html file
fig.write_html("../Charts/authoer_with_most_books.html")
fig.show()

'Agatha Christie', 'William Shakespeare' and 'Stephen King' are the authors who had written most books

* Which year have most books published

In [ ]:
temp_df=book_df['Year-Of-Publication'].value_counts().iloc[:20]

fig = go.Figure()
fig.add_bar(x=temp_df.index,
            y=temp_df.values,
            text=temp_df.values,
            hovertemplate ='Year: %{x}<br>Total Books: %{y}<extra></extra>',
            marker=dict(color=temp_df.values,
                        colorscale='Tropic'),
            textposition='outside',
            marker_line_width=1.,
            width=0.6,
            marker_line_color='#b330ff',)
fig.update_layout(template="plotly_white",
                  title='<b>The year which had the most no of books published,<b>',
                  xaxis=dict(
                            title='Year',
                            tickvals=temp_df.index,
                            linecolor='#728082',
                            ticks='outside'),
                  yaxis=dict(
                            showgrid=False,
                            title='No of Books',
                            linecolor='#728082',
                            ticks='outside'),)
# saving as html file
fig.write_html("../Charts/year_when_most_books.html")
fig.show()

After the Year '1999' till '2002' same no of books were published also thses are heighest no of all time.


* Lets check with different centuries

In [ ]:
fig = go.Figure()
temp_df=book_df['Year-Of-Publication'].dropna().apply(lambda x : str(int(x))[:2]).value_counts()
fig.add_bar(y=[f'{int(x)}<sup>th</sup>' for x in temp_df.index],
            x=temp_df.values,
            text=temp_df.values,
            orientation='h',
            hovertemplate ='Books Count:<br>%{x} <extra></extra>',
            marker=dict(color=temp_df.values,
                        colorscale='Plotly3'),
            textposition='outside',
            marker_line_width=1.,
            width=0.6,
            marker_line_color='#91d2fa',)
fig.update_layout(template="plotly_white",
                  title='<b>Books published in different centuries</b>',
                  xaxis=dict(
                            showgrid=False,
                            title='No of Books published',
                            linecolor='#728082',
                            ticks='outside'),
                  yaxis=dict(
                            title='Centuries',
                            linecolor='#728082',
                            ticks='outside'),)
# saving as html file
fig.write_html("../Charts/centuries_with_most_books.html")
fig.show()

As per the data sets we have, can see 19<sup>th</sup> centuries is there year when must books are published.

* Which publisher had published most no of books

In [ ]:
# Checking 'publisher' column
book_df['Publisher']

In [ ]:
# '&Amp' is a character reference for 'Ampersand'
book_df[book_df['Publisher'].apply(lambda x: True if '&Amp;' in x else False)]


In [ ]:
# We can replace them with '&'
book_df['Publisher']=book_df['Publisher'].apply(lambda x: x.replace('&Amp;','&'))

In [ ]:
# the graph

temp_df=book_df['Publisher'].value_counts()[:20]

fig = go.Figure()
fig.add_bar(x=temp_df.index,
            y=temp_df.values,
            text=temp_df.index,
            hovertemplate ='Total Books:<br> %{y}<extra></extra>',
            marker=dict(color=temp_df.values,
                        colorscale='Picnic'),
            marker_line_width=1.,
            width=0.6,
            marker_line_color='#02b0fa',)
annotations=[dict(x=val[0], y=val[1]+200, text=str(val[1]), showarrow=False) for val in zip(temp_df.index, temp_df.values)]
fig.update_layout(annotations=annotations,
                  template="plotly_white",
                  title='<b>Publisher who published most no of books<b>',
                  margin=dict(t=65),
                  xaxis=dict(
                            showgrid=False,
                            title='Publishers',
                            linecolor='#728082',
                            showticklabels=False,),
                  yaxis=dict(
                            showgrid=False,
                            title='No of Books',
                            linecolor='#728082',
                            ticks='outside'),)
# saving as html file
fig.write_html("../Charts/Publisher_with_most_books.html")
fig.show()

'Harlequin' is the Publisher where most no of books are published

#### 2. Users_df

In [ ]:
users_df.head()

In [ ]:
# We dont need User-ID column as it same as index column

users_df.drop('User-ID',inplace=True,axis=1)

In [ ]:
# Checking data types
users_df.info()

In [ ]:
# Checking if any null values
count_null_value(users_df)

In [ ]:
sn.heatmap(users_df.isnull(),cbar=False)

In [ ]:
# visualizing how much null values we have
null_counts_pie_chart(users_df['Age'])

In [ ]:
# So many users refused to give their personal details
print(f"{np.around(110762/len(users_df)*100,2)}% of people denied to share their 'Age'")

In [ ]:
# Checking numeric counts and non numeric counts
show_counts(users_df)

In [ ]:
# Checking the distribution of Buyers 'Age'
sn.histplot(users_df['Age'],kde=True,color='#1fba04')

People's age are crossing 200!!<br>
Are they purchasing from Heaven?

In [ ]:
# most children learn to read by age 6 or 7.
# Also olds are dying by age 90
users_df[(users_df['Age']<7) | (users_df['Age']>90) ]=np.nan

    * Which age group is reading more books

In [ ]:
# Making a new column named 'User_groups'
users_df['User_groups']=users_df['Age'].apply(lambda x: 'Children' if x <15 else 'Youth' if x <25 else 'Adult' if x <65 else 'Seniors')

In [ ]:
# Visualizing it
temp_df=users_df['User_groups'].value_counts()
colors=['lightcyan','cyan','royalblue','darkblue']
fig=go.Figure(go.Pie(labels=temp_df.index,
                     values=temp_df.values,
                    #  pull=[0.01 for i in range(4)],
                     marker=dict(colors=colors)))
fig.update_layout(title="Different age group and reading book",
                    autosize=False,)
# saving as html file
fig.write_html("../Charts/age_group_reading_books.html")
fig.show()

As we can see, Adults and Seniors are reading more number of books

In [ ]:
# Now checking the null values percentage of 'Age' column
null_counts_pie_chart(users_df['Age'])

In [ ]:
print(f"Around {users_df['Age'].isnull().sum()-110762} users were miss typed their age.")

In [ ]:
# Lets see after imputing null values with mean values

plt.figure(figsize=(16,5))
temp_df=users_df['Age'].dropna()
plt.subplot(1,2,1)
sn.histplot(temp_df,kde=True,label=f"skewness: {stats.skew(temp_df):.2f}\nKurtosis: {stats.kurtosis(temp_df):.2f}")
plt.legend()
temp_df=users_df['Age'].fillna(users_df['Age'].mean())
plt.subplot(1,2,2)
sn.histplot(temp_df,kde=True,label=f"With Mean value Imputation\nskewness: {stats.skew(temp_df):.2f}\nKurtosis: {stats.kurtosis(temp_df):.2f}")
plt.legend()

Do you think its a good idea to replace null values with 'mean'<br>
as it significantly changes the kurtosis values.
Even replacing with median values is also not a good idea <br>as we have around 40% data are missing.

Those red points are newly generated points.<br>
We have more than 2.5 lakhs of datas so its hard to visualize them

In [ ]:
# Lets see with histogram

plt.figure(figsize=(16,5))
temp_df=users_df['Age'].dropna()
plt.subplot(1,2,1)
sn.histplot(temp_df,kde=True,label=f"skewness: {stats.skew(temp_df):.2f}\nKurtosis: {stats.kurtosis(temp_df):.2f}")
plt.legend()
temp_df=users_df['Age'].ffill().dropna()
plt.subplot(1,2,2)
sn.histplot(temp_df,kde=True,label=f"pad Interpolation\nskewness: {stats.skew(temp_df):.2f}\nKurtosis: {stats.kurtosis(temp_df):.2f}")
plt.legend()

In [ ]:
# Now replacing mean with pad interpolated values
users_df['Age'].ffill(inplace=True)

* which country has the most readers

In [ ]:
# Checking the length of values given by each user
users_df['Location'].apply(lambda x: len(str(x).split(','))).value_counts()

In [ ]:
# Need to clean country column
users_df['Location']=users_df['Location'].apply(clean_location)

In [ ]:
# again checking length of values
users_df['Location'].apply(lambda x:len(x)).value_counts()

In [ ]:
# creating required columns
users_df[['Dist','State','Country']]=""

In [ ]:
# we can make three column specifing dist,state,country
# Also if a row doesn't have all three information we will replace them with nan

for i in range(1,4):
    users_df.iloc[:,-i]=users_df['Location'].apply(lambda x : x[-i] if len(x)>=i else np.nan)

In [ ]:
# Checking how much null values we have
sn.heatmap(users_df.isnull(),cbar=False)

In [ ]:
# Seems like most users have given last word as the country name

temp_df=users_df['Country'].value_counts().iloc[:10]

# The figure
fig = go.Figure()
fig.add_bar(x=temp_df.values,
            y=temp_df.index,
            text=temp_df.values,
            orientation='h',
            hovertemplate ='Total Book Readers:<br>%{x} <extra></extra>',
            marker=dict(color=temp_df.values,
                        colorscale='oxy'),
            textposition='outside',
            marker_line_width=1.,
            width=0.6,
            marker_line_color='#ff61fa',)
fig.update_layout(template="plotly_white",
                  title='<b>Users from different country</b>',
                  xaxis=dict(
                            showgrid=False,
                            title='Total users reviewed ',
                            linecolor='#728082',
                            ticks='outside'),
                  yaxis=dict(
                            title='Countries',
                            linecolor='#728082',
                            ticks='outside'),)
fig.show()

# saving as html file
fig.write_html("../Charts/users_from_different_country.html")

'USA' has the most readers compared to other countries folowed by 'Canada' and 'UK'.

#### 3. Ratings _df

In [ ]:
ratings_df.head()

In [ ]:
ratings_df.info()

In [ ]:
# Checking null values
count_null_value(ratings_df)

* Which user had given most ratings

In [ ]:
temp_df=ratings_df['User-ID'].value_counts().iloc[:20]

fig = go.Figure()
fig.add_bar(x=[str(x) for x in temp_df.index],
            y=temp_df.values,
            text=temp_df.values,
            hovertemplate ='User ID: %{x}<br>Total Books: %{y}<extra></extra>',
            marker=dict(color=temp_df.values,
                        colorscale='Geyser'),
            textposition='outside',
            marker_line_width=1.,
            width=0.6,
            marker_line_color='#d9ff03',)
fig.update_layout(template="plotly_white",
                  title='<b>User who rated most no of books<b>',
                  xaxis=dict(
                            title='User-ID',
                            tickvals=temp_df.index,
                            linecolor='#728082',
                            ticks='outside'),
                  yaxis=dict(
                            showgrid=False,
                            title='No of Books',
                            linecolor='#728082',
                            ticks='outside'),)
fig.show()

# saving as html file
fig.write_html("../Charts/user_id_with_given_ratings.html")

* ratings count

In [ ]:
temp_df=ratings_df['Book-Rating'].value_counts().sort_index()
fig = go.Figure()
fig.add_bar(y=temp_df.index,
            x=temp_df.values,
            text=temp_df.values,
            orientation='h',
            hovertemplate ='Reviewer:<br>%{x} <extra></extra>',
            marker=dict(color=temp_df.values,
                        colorscale='Rainbow'),
            textposition='outside',
            marker_line_width=1.,
            width=0.6,
            marker_line_color='#97ff91',)
fig.update_layout(template="plotly_white",
                  title='<b>Total Ratings</b>',
                  xaxis=dict(
                            showgrid=False,
                            title='Users',
                            linecolor='#728082',
                            ticks='outside'),
                  yaxis=dict(
                            title='Ratings',
                            tickvals=temp_df.index,
                            linecolor='#728082',
                            ticks='outside'),)
fig.show()

# saving as html file
fig.write_html("../Charts/most_given_ratings.html")

In most cases, we consider 0 review means that the user have not given any rating.

In [ ]:
# So removing all 0 rated row and storing to new dataframe
new_ratings_df=ratings_df.copy()

new_ratings_df.drop(ratings_df[ratings_df['Book-Rating']==0].index,inplace=True)

In [ ]:
new_ratings_df.head()

In [ ]:
# Making every values of ISBN to upper case, also removing extra spaces
new_ratings_df['ISBN']=new_ratings_df['ISBN'].apply(lambda x: x.strip().upper())

In [ ]:
# length of ISBN should be of length 10 right!
new_ratings_df['ISBN'].apply(lambda x: len(x)).value_counts()

In [ ]:
# Can see some of them are different from 10, what are they?
new_ratings_df[new_ratings_df['ISBN'].apply(lambda x: len(x))!=10]['ISBN'].unique()

In [ ]:
# But ISBN only contains numbers and X
new_ratings_df['ISBN']=new_ratings_df['ISBN'].apply(lambda x:re.sub('[^0-9X]','',x))
new_ratings_df.head()

In [ ]:
new_ratings_df[new_ratings_df['ISBN'].apply(lambda x: len(x)!=10)].index

In [ ]:
# Dropping all ISBN's which is not of length 10

temp_index=new_ratings_df[new_ratings_df['ISBN'].apply(lambda x: len(x)!=10)].index
new_ratings_df.drop(index=temp_index,inplace=True)

In [ ]:
new_ratings_df.info()

In [ ]:
# Calculating avg rating and total no of reviewers
avg_rating_df=new_ratings_df.groupby('ISBN').agg({'Book-Rating':[np.mean,np.size]})

In [ ]:
avg_rating_df.columns=avg_rating_df.columns.map('-'.join)

In [ ]:
avg_rating_df=avg_rating_df.reset_index().rename(columns={'Book-Rating-size':'Total-Reviewers','Book-Rating-mean':'Book-Rating'})

Imagine a book has rated 10 but total reviewer is 1. Can we consider this a popular book?<br>
We must specify a criteria to no of reviewers should be greater than a specific no.

In [ ]:
avg_rating_df

In [ ]:
# restricting no of reviewers to 10
avg_rating_df=avg_rating_df[avg_rating_df['Total-Reviewers']>10]
avg_rating_df.head()

In [ ]:
# Checking info
avg_rating_df.info()

Man! we have only 512 no of rows left.<br>

One of the Achievements of Data Scientist😹, agree?🤟

In [ ]:
# merging avg_rating_df with book_df on ISBN
mer_rat_book=book_df.merge(avg_rating_df,on='ISBN')
mer_rat_book.head(2)

In [ ]:
# dropping small and medium image URLs as we dont need them
mer_rat_book.drop(['Image-URL-S','Image-URL-M'],axis=1,inplace=True)

In [ ]:
# keeping decimal points of Rating to 2
mer_rat_book['Book-Rating']=mer_rat_book['Book-Rating'].apply(lambda x: np.around(x,2))

In [ ]:
# making Year-Of-Publication to int16 data type
mer_rat_book['Year-Of-Publication']=mer_rat_book['Year-Of-Publication'].astype('int16')

In [ ]:
# There may be some column which is repeated
# So dropping them
mer_rat_book.drop_duplicates(subset='ISBN',inplace=True)

* So which is the most popular books

On an average if rating is more than 7 and<br>
also more than 100 peoples are given rating,<br>
then we can say its a popular book.

In [ ]:
# restricting ratings to 7 and reviewers to 250 also sorting them by Book-Rating

temp_df=mer_rat_book[(mer_rat_book['Total-Reviewers']>250)&(mer_rat_book['Book-Rating']>7)].sort_values(by='Book-Rating',ascending=False).iloc[:10]


fig=go.Figure()
fig.add_bar(y=temp_df.iloc[:,1],
            x=temp_df.iloc[:,-2],
            customdata=temp_df.iloc[:,-1],
            text=temp_df.iloc[:,-2],
            orientation='h',
            width = 0.5,
            hovertemplate ='Ratings: %{x}<br>Reviewers: %{customdata} <extra></extra>',
            marker=dict(color=temp_df.iloc[:,-2],
                        colorscale='Sunset'),
            textposition='inside',
            marker_line_width=1.,
            marker_line_color='#b577fc',)

fig.update_layout(template="plotly_white",
                  title='<b>Famous Books</b>',
                #   margin=dict(r=200),
                  height=500,
                  width=1000,
                  xaxis=dict(
                            showgrid=False,
                            title='Average Ratings',
                            linecolor='#728082',
                            ticks='outside'),
                  yaxis=dict(
                            linecolor='#728082',
                            ticks='outside'),)

for i in range(10):
    fig.add_layout_image(dict(
                              source=extract_image(temp_df.iloc[i,0],book_df),
                              x=temp_df.iloc[i,-2]+ 0.1, y=0.09+0.1*i,
                              xref='x', yref='paper',
                              sizex=.3, sizey=.3))
fig.show()

# saving as html file
fig.write_html("../Charts/famous_books.html")

In [ ]:
# Dumping necessary csv files so as to built recommender system
mer_rat_book.to_csv('../data/processed/mer_rat_book.csv.gz', index=False, compression='gzip')
new_ratings_df.to_csv('../data/processed/new_ratings_df.csv', index=False)
avg_rating_df.to_csv('../data/processed/avg_rating_df.csv', index=False)
book_df.to_csv('../data/processed/book_df_cleaned.csv.gz', index=False,compression='gzip')